***App Testing run and generation of file with App Output***

In [1]:
%run ../Setup.ipynb

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


**Testing App 1b**

In [2]:
%run ../A_Apps/1b_Openai_RAG_Schema.ipynb

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Retrieved Documents:
page_content='SELECT T1.artist_name ,  count(*) FROM artist AS T1 JOIN song AS T2 ON T1.artist_name  =  T2.artist_name WHERE T2.languages  =  "english" GROUP BY T2.artist_name HAVING count(*)  >=  1' metadata={'_id': '66cf12c13c2173e47d7af5dd', 'Question': 'Find the names and number of works of all artists who have at least one English songs.'}
page_content='SELECT T1.artist_name ,  count(*) FROM artist AS T1 JOIN song AS T2 ON T1.artist_name  =  T2.artist_name WHERE T2.languages  =  "english" GROUP BY T2.artist_name HAVING count(*)  >=  1' metadata={'_id': '66cf12c13c2173e47d7af5de', 'Question': 'What are the names and number of works for all artists who have sung at least one song in English?'}
page_content='SELECT count(DISTINCT title) FROM vocals AS T1 JOIN songs AS T2 ON T1.songid  =  T2.songid WHERE TYPE  =  "shared"' metadata={

In [3]:
# Upload the dataset and transform to dataframe
# Define the dataset path
dataset_path = "Spider_Testing_Selection.csv"
print("Dataset Path:", dataset_path)

# Check if the file exists at the specified path
if not os.path.isfile(dataset_path):
    raise FileNotFoundError(f"Unable to find the file at {dataset_path}")

# Load the dataset
testing_1b = load_dataset('csv', data_files=dataset_path)

# Convert the dataset to a pandas dataframe
df_1b_testing = testing_1b["train"].to_pandas()

# Print a few rows to verify
print(df_1b_testing.head())

Dataset Path: Spider_Testing_Selection.csv
            DB_ID                                              Query  \
0  concert_singer  SELECT T2.name ,  T2.capacity FROM concert AS ...   
1          pets_1  SELECT T1.fname ,  T1.age FROM student AS T1 J...   
2           car_1  SELECT T1.CountryName FROM COUNTRIES AS T1 JOI...   
3           car_1  SELECT T2.MakeId ,  T2.Make FROM CARS_DATA AS ...   
4           car_1  select t1.id ,  t1.maker from car_makers as t1...   

                                            Question  
0  Show the stadium name and capacity with most n...  
1  Find the first name and age of students who ha...  
2  Which countries in europe have at least 3 car ...  
3  Among the cars with more than lowest horsepowe...  
4  Which are the car makers which produce at leas...  


In [4]:
# Function to run the chain for each question
def process_questions(df_1b_testing):
    # Create an empty list to store the results
    output = []

    for i, row in df_1b_testing.iterrows():
        # Get the question from the dataframe
        question = row["Query"]

        # Execute the chain with the current question
        try:
            result = chain_1b.invoke(question)
        except Exception as e:
            result = f"Error processing question {i}: {str(e)}"
        
        # Append the result to the list
        output.append(result)

    # Add the results to a new column in the dataframe
    df_1b_testing["Output"] = output
    
# Check and split the Output column into two: Translation and Explanation
    def split_output(text):
        if 'Explanation:' in text:
            parts = text.split('Explanation:', 1)
            return parts[0].strip(), parts[1].strip()
        else:
            return text, None  # If no "Explanation:", return the text as translation, and None for explanation

    # Apply the splitting function to the Output column
    df_1b_testing[['Translation', 'Explanation']] = df_1b_testing["Output"].apply(lambda x: pd.Series(split_output(x)))
    
    return df_1b_testing

# Call the function and process the dataframe
df_1b_testing_output = process_questions(df_1b_testing)

# Now 'df_with_results' contains the original questions and their corresponding results
print(df_1b_testing_output)
df_1b_testing_output.to_csv("Testing_Output_1b.csv", index=False)

Retrieved Documents:
page_content='SELECT t1.name FROM stadium AS t1 JOIN event AS t2 ON t1.id  =  t2.stadium_id GROUP BY t2.stadium_id ORDER BY count(*) DESC LIMIT 1' metadata={'_id': '66cf12c13c2173e47d7afdbc', 'Question': 'What is the name of the stadium which held the most events?'}
page_content='SELECT t3.name FROM record AS t1 JOIN event AS t2 ON t1.event_id  =  t2.id JOIN stadium AS t3 ON t3.id  =  t2.stadium_id GROUP BY t2.stadium_id ORDER BY count(*) DESC LIMIT 1' metadata={'_id': '66cf12c13c2173e47d7afdc5', 'Question': 'Find the names of stadiums that the most swimmers have been to.'}
page_content='SELECT t2.firstname FROM Performance AS t1 JOIN Band AS t2 ON t1.bandmate  =  t2.id JOIN Songs AS T3 ON T3.SongId  =  T1.SongId GROUP BY firstname ORDER BY count(*) DESC LIMIT 1' metadata={'_id': '66cf12c13c2173e47d7afc52', 'Question': 'Find the first name of the band mate that has performed in most songs.'}
page_content='SELECT t2.firstname FROM Performance AS t1 JOIN Band AS t2 O

**Testing App 2b**

In [5]:
%run ../A_Apps/2b_Openai_RAG.ipynb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Retrieved Schema:
page_content='SINGER_ID SINGER ' metadata={'_id': '66e83eb4203411a6471d680a', 'Column_name': 'SINGER_ID', 'Table_name': 'SINGER ', 'DB_name': 'singer'}
page_content='SINGER_ID SINGER ' metadata={'_id': '66e83eb4203411a6471d67a5', 'Column_name': 'SINGER_ID', 'Table_name': 'SINGER ', 'DB_name': 'concert_singer'}
page_content='SINGER_ID SINGER_IN_CONCERT ' metadata={'_id': '66e83eb4203411a6471d67b5', 'Column_name': 'SINGER_ID', 'Table_name': 'SINGER_IN_CONCERT ', 'DB_name': 'concert_singer'}
page_content='SINGER_ID SONG ' metadata={'_id': '66e83eb4203411a6471d6812', 'Column_name': 'SINGER_ID', 'Table_name': 'SONG ', 'DB_name': 'singer'}
page_content='CONCERT_ID SINGER_IN_CONCERT ' metadata={'_id': '66e83eb4203411a6471d67b4', 'Column_name': 'CONCERT_ID', 'Table_name': 'SINGER_I

In [6]:
# Upload the dataset and transform to dataframe
# Define the dataset path
dataset_path = "Spider_Testing_Selection.csv"
print("Dataset Path:", dataset_path)

# Check if the file exists at the specified path
if not os.path.isfile(dataset_path):
    raise FileNotFoundError(f"Unable to find the file at {dataset_path}")

# Load the dataset
testing_2b = load_dataset('csv', data_files=dataset_path)

# Convert the dataset to a pandas dataframe
df_2b_testing = testing_2b["train"].to_pandas()

# Print a few rows to verify
print(df_2b_testing.head())

Dataset Path: Spider_Testing_Selection.csv
            DB_ID                                              Query  \
0  concert_singer  SELECT T2.name ,  T2.capacity FROM concert AS ...   
1          pets_1  SELECT T1.fname ,  T1.age FROM student AS T1 J...   
2           car_1  SELECT T1.CountryName FROM COUNTRIES AS T1 JOI...   
3           car_1  SELECT T2.MakeId ,  T2.Make FROM CARS_DATA AS ...   
4           car_1  select t1.id ,  t1.maker from car_makers as t1...   

                                            Question  
0  Show the stadium name and capacity with most n...  
1  Find the first name and age of students who ha...  
2  Which countries in europe have at least 3 car ...  
3  Among the cars with more than lowest horsepowe...  
4  Which are the car makers which produce at leas...  


In [7]:
# Function to run the chain for each question
def process_questions(df_2b_testing):
    # Create an empty list to store the results
    output = []

    for i, row in df_2b_testing.iterrows():
        # Get the question from the dataframe
        question = row["Query"]

        # Execute the chain with the current question
        try:
            result = chain_2b.invoke(question)
        except Exception as e:
            result = f"Error processing question {i}: {str(e)}"
        
        # Append the result to the list
        output.append(result)

    # Add the results to a new column in the dataframe
    df_2b_testing["Output"] = output
    
# Check and split the Output column into two: Translation and Explanation
    def split_output(text):
        if 'Explanation:' in text:
            parts = text.split('Explanation:', 1)
            return parts[0].strip(), parts[1].strip()
        else:
            return text, None  # If no "Explanation:", return the text as translation, and None for explanation

    # Apply the splitting function to the Output column
    df_2b_testing[['Translation', 'Explanation']] = df_2b_testing["Output"].apply(lambda x: pd.Series(split_output(x)))
    
    return df_2b_testing

# Call the function and process the dataframe
df_2b_testing_output = process_questions(df_2b_testing)

# Now 'df_with_results' contains the original questions and their corresponding results
print(df_2b_testing_output)
df_2b_testing_output.to_csv("Testing_Output_2b.csv", index=False)

Retrieved Schema:Retrieved Documents:
page_content='SELECT t1.name FROM stadium AS t1 JOIN event AS t2 ON t1.id  =  t2.stadium_id GROUP BY t2.stadium_id ORDER BY count(*) DESC LIMIT 1' metadata={'_id': '66cf12c13c2173e47d7afdbc', 'Question': 'What is the name of the stadium which held the most events?'}
page_content='SELECT t3.name FROM record AS t1 JOIN event AS t2 ON t1.event_id  =  t2.id JOIN stadium AS t3 ON t3.id  =  t2.stadium_id GROUP BY t2.stadium_id ORDER BY count(*) DESC LIMIT 1' metadata={'_id': '66cf12c13c2173e47d7afdc5', 'Question': 'Find the names of stadiums that the most swimmers have been to.'}
page_content='SELECT t2.firstname FROM Performance AS t1 JOIN Band AS t2 ON t1.bandmate  =  t2.id JOIN Songs AS T3 ON T3.SongId  =  T1.SongId GROUP BY firstname ORDER BY count(*) DESC LIMIT 1' metadata={'_id': '66cf12c13c2173e47d7afc52', 'Question': 'Find the first name of the band mate that has performed in most songs.'}
page_content='SELECT t2.firstname FROM Performance AS t1 

**Testing App 3b**

In [10]:
%run ../A_Apps/3b_Openai.ipynb


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [11]:
# Upload the dataset and transform to dataframe
# Define the dataset path
dataset_path = "Spider_Testing_Selection.csv"
print("Dataset Path:", dataset_path)

# Check if the file exists at the specified path
if not os.path.isfile(dataset_path):
    raise FileNotFoundError(f"Unable to find the file at {dataset_path}")

# Load the dataset
testing_3b = load_dataset('csv', data_files=dataset_path)

# Convert the dataset to a pandas dataframe
df_3b_testing = testing_3b["train"].to_pandas()

# Print a few rows to verify
print(df_3b_testing.head())

Dataset Path: Spider_Testing_Selection.csv
            DB_ID                                              Query  \
0  concert_singer  SELECT T2.name ,  T2.capacity FROM concert AS ...   
1          pets_1  SELECT T1.fname ,  T1.age FROM student AS T1 J...   
2           car_1  SELECT T1.CountryName FROM COUNTRIES AS T1 JOI...   
3           car_1  SELECT T2.MakeId ,  T2.Make FROM CARS_DATA AS ...   
4           car_1  select t1.id ,  t1.maker from car_makers as t1...   

                                            Question  
0  Show the stadium name and capacity with most n...  
1  Find the first name and age of students who ha...  
2  Which countries in europe have at least 3 car ...  
3  Among the cars with more than lowest horsepowe...  
4  Which are the car makers which produce at leas...  


In [12]:
# Function to run the chain for each question
def process_questions(df_3b_testing):
    # Create an empty list to store the results
    output = []

    for i, row in df_3b_testing.iterrows():
        # Get the question from the dataframe
        question = row["Query"]

        # Execute the chain with the current question
        try:
            result = chain_3b.invoke(question)
        except Exception as e:
            result = f"Error processing question {i}: {str(e)}"
        
        # Append the result to the list
        output.append(result)

    # Add the results to a new column in the dataframe
    df_3b_testing["Output"] = output
    
# Check and split the Output column into two: Translation and Explanation
    def split_output(text):
        if 'Explanation:' in text:
            parts = text.split('Explanation:', 1)
            return parts[0].strip(), parts[1].strip()
        else:
            return text, None  # If no "Explanation:", return the text as translation, and None for explanation

    # Apply the splitting function to the Output column
    df_3b_testing[['Translation', 'Explanation']] = df_3b_testing["Output"].apply(lambda x: pd.Series(split_output(x)))
    
    return df_3b_testing

# Call the function and process the dataframe
df_3b_testing_output = process_questions(df_3b_testing)

# Now 'df_with_results' contains the original questions and their corresponding results
print(df_3b_testing_output)
df_3b_testing_output.to_csv("Testing_Output_3b.csv", index=False)

                           DB_ID  \
0                 concert_singer   
1                         pets_1   
2                          car_1   
3                          car_1   
4                          car_1   
5                          car_1   
6                          car_1   
7                       flight_2   
8                       flight_2   
9       employee_hire_evaluation   
10      employee_hire_evaluation   
11          cre_Doc_Template_Mgt   
12                  course_teach   
13                  museum_visit   
14                  museum_visit   
15                         wta_1   
16  student_transcripts_tracking   
17  student_transcripts_tracking   
18                       voter_1   
19                       world_1   
20                       world_1   
21                       world_1   
22                       world_1   
23                       world_1   
24                       world_1   
25                     orchestra   
26                   dog_ken